In [1]:
import re
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
# ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
# ADDRESS = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86"
ADDRESS = "0x6c2a355929ee1262305e385ad49b84fe5f5a4777"

START_DATE = "2024-05-01 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2025-01-01 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE", "DAI"}
base_pattern = re.compile('|'.join(BASE_TOKENS))

In [ ]:
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, base_pattern)
# 这里我为了显示简洁把process_transactions中print结果的注释掉了 如果需要可以取消注释

2025-01-15 16:57:25,966 - INFO - Start block: 19771559, End block: 21525891
2025-01-15 16:57:25,966 - INFO - Fetching ERC20 transfers from block 19771559 to 21525891...
2025-01-15 16:57:27,352 - INFO - Page 1: Retrieved 1000 transactions.
2025-01-15 16:57:28,735 - INFO - Page 2: Retrieved 1000 transactions.
2025-01-15 16:57:30,120 - INFO - Page 3: Retrieved 1000 transactions.
2025-01-15 16:57:31,637 - INFO - Page 4: Retrieved 1000 transactions.
2025-01-15 16:57:33,040 - INFO - Page 5: Retrieved 1000 transactions.
2025-01-15 16:57:34,523 - INFO - Page 6: Retrieved 1000 transactions.
2025-01-15 16:57:35,908 - INFO - Page 7: Retrieved 1000 transactions.
2025-01-15 16:57:37,793 - INFO - Page 8: Retrieved 1000 transactions.
2025-01-15 16:57:39,177 - INFO - Page 9: Retrieved 1000 transactions.
2025-01-15 16:57:40,562 - INFO - Page 10: Retrieved 1000 transactions.
2025-01-15 16:57:41,429 - WARNING - API request failed on page 11: Request failed: Result window is too large, PageNo x Offset siz

...Merging 3003 groups of transactions...
Transaction Hash 0x00c5efc959654417d5ace350856ee4e0ccbfbb8cb0c307dadb5d0c389e968e02 contains 4 records:
Transaction Hash 0x00cd2a41678a5b7aae4c1e97a80b4865411c19965235d5500e2ebc9c3238acf9 contains 4 records:
Transaction Hash 0x00e1b40f23ffc2d0967a96cac8a1cb4de92e7ab4e7f2617eabd8a512db2f4945 contains 10 records:
Transaction Hash 0x011169c6efee55b084b0d71443ec65b0d728def27ac5aae803f2d6d07ab16133 contains 4 records:
Transaction Hash 0x01f5554e71c97b1a51b43b64ccceeaef2039e1b034f75a0bb2d3f01dc68dbd3f contains 8 records:
Transaction Hash 0x052ca792656135f686258d095a1cccb03c9b1d8ad6110a890bb84ad8c7e18d13 contains 4 records:
Transaction Hash 0x05afc7463a71669cd8fffe7cc05cd69e7a5ec77e0e5a276cbcdeffac84a2f8eb contains 8 records:
Transaction Hash 0x0611d77a4857c12e708edf1d7dcc164b21282f5c49fb0110042d2e8af4932401 contains 6 records:
Transaction Hash 0x06315896fd8b9ed4f5235cacd7437fe02c9adfaf6a9b0fd43d1a4ea9d1f7c84b contains 4 records:
Transaction Hash 0x06

In [5]:
# 计算pnl
if not record_df_1.empty:
    unique_tokens_count = record_df_1["token_symbol"].nunique()
    unique_tokens = record_df_1["token_symbol"].unique()
    results = []
    # 在USDC USDE USDT DAI的价格默认为1：1的情况下，不考虑不同的稳定币作为单位的情况

    for token in unique_tokens:
        token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
        unique_price_units = token_df["price_unit"].unique()
        price_units_str = ", ".join(unique_price_units)
        
        print(f"{token} (Price Units: {price_units_str}): {len(token_df)} records")

        pnl_df = calculate_pnl(token_df)
        print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
        total_pnl = pnl_df["pnl"].sum()

        results.append({"token": token, "price_unit": price_units_str, "total_pnl": total_pnl})

    pnl_summary_df = pd.DataFrame(results)
    print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
    
    # # 将结果保存为 CSV 文件
    # pnl_summary_df.to_csv("pnl_summary.csv", index=False)
    # print("PNL summary saved to pnl_summary.csv")

    # # 如果需要将每个 token 的 PNL 详情也保存为 CSV
    # for token in unique_tokens:
    #     token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
    #     pnl_df = calculate_pnl(token_df)
    #     pnl_df.to_csv(f"pnl_details_{token}.csv", index=False)
    #     print(f"PNL details for {token} saved to pnl_details_{token}.csv")
          
else:
    print("No transactions found")

WETH (Price Units: USDT/WETH, aEthUSDT/WETH, DAI/WETH, aEthDAI/WETH): 997 records
+---------------------+-------------+--------+----------+---------+------------+----------------+
| Date                |   timeStamp | Type   |   amount |   price |        pnl |   net_position |
+=====================+=============+========+==========+=========+============+================+
| 2024-09-29 15:07:11 |  1727622431 | BUY    |   226.00 | 2654.89 |  168392.72 |         226.00 |
+---------------------+-------------+--------+----------+---------+------------+----------------+
| 2024-09-29 15:07:11 |  1727622431 | BUY    |   629.40 | 2654.89 |  468973.72 |         855.40 |
+---------------------+-------------+--------+----------+---------+------------+----------------+
| 2024-09-29 15:07:11 |  1727622431 | BUY    |    74.58 | 2654.89 |   55569.60 |         929.98 |
+---------------------+-------------+--------+----------+---------+------------+----------------+
| 2024-09-29 15:07:23 |  1727622443 

In [6]:
TOP_ACCOUNTS_DF = pd.read_csv(OUTPUT_FILE + "/top_accounts.csv")
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
output_file = OUTPUT_FILE + "/address_counts_with_name_tags.csv"

process_and_save_address(transfers_df, TOP_ACCOUNTS_DF, output_file)

100%|██████████| 72/72 [01:41<00:00,  1.40s/it]

Data has been save to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result/address_counts_with_name_tags.csv


In [7]:
address_counts = pd.read_csv(OUTPUT_FILE + "/address_counts_with_name_tags.csv")
address_threshold = 10
important_addresses = address_counts[address_counts["total_value_count"] > address_threshold]
important_addresses = important_addresses.sort_values(by="total_value_count", ascending=False)
# important_addresses = important_addresses[important_addresses["address"] != ADDRESS.lower()]

print(important_addresses)


                                       address  from_count  to_count  \
33  0x6c2a355929ee1262305e385ad49b84fe5f5a4777        4505      5495   
1   0x00000000009e50a7ddb7a7b0e2ee6604fd120e49        1019      1055   
0   0x0000000000000000000000000000000000000000        1749        21   
16  0x3face000cb537ad065b90ef47f23d62b48b2264c         549       531   
67  0xe6b1de575e7e610889ea21024834e120f92033a3         492       496   
47  0x9d6b911199b891c55a93e4bc635bf59e33d002d8         245       275   
39  0x81463b0f960f247f704377661ec81c1fd65b5128         231       275   
30  0x64545160d28fd0e309277c02d6d73b3923cc4bfa         158       161   
65  0xe08d97e151473a848c3d9ca3f323cb720472d015         136       141   
50  0xa9048585166f4f7c4589ade19567bb538035ed36         140       133   
2   0x00000688768803bbd44095770895ad27ad6b0d95          84       186   
56  0xc37704a457b1ee87eb657cae584a34961e86acac          89        89   
49  0xa8c1c98aaf99a5dfc907d61b892b2ad624901185          68      

In [ ]:
import sys
import os
import getpass
# find a way to use relative path, so that the function in dynamoUtil can be run in any machine
src_path = r'C:\Users\YuweiCao\Documents\GitHub\char-python-common\src'

if src_path not in sys.path:
    sys.path.append(src_path)

USER = getpass.getuser()
REPO_PATH = r'C:\Users\YuweiCao\Documents\GitHub\char-python-common'

sys.path.append(REPO_PATH)
sys.path.append(REPO_PATH + '/src')

from dynamoUtil import price_bar_query
from datetime import datetime, timedelta

mapping = {
    'ETH': 'ETHUSDT-OKX-00000000-FUT',
    'SOL': 'SOLUSDT-OKX-00000000-FUT',
    'BTC': 'BTCUSDT-OKX-00000000-FUT',
}

def fetch_price_data(chain_name):
    today = datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
    start_date = today - timedelta(days=365) # 一年的数据

    today_str = today.strftime('%Y-%m-%d-%H-%M')
    start_date_str = start_date.strftime('%Y-%m-%d-%H-%M')
    
    instrument_code = mapping.get(chain_name)

    price_data = price_bar_query(instrument_code + '|1440', start_date_str, today_str, 'live')
    price_df = pd.DataFrame(price_data)
    print(f"Columns in DataFrame for {chain_name}: {price_df.columns}") 
    print(price_df.head())
    
    if price_df.empty:
        print(f"No valid data found for {chain_name}. Skipping.")
        return

    if 'timestamp' not in price_df.columns or 'close' not in price_df.columns:
        print(f"Required columns 'timestamp' or 'close' not found in the data for {chain_name}. Skipping.")
        return

    selected_columns = price_df[['timestamp', 'close']].copy()

    # selected_columns['date'] = pd.to_datetime(selected_columns['timestamp']).dt.date

    selected_columns.rename(columns={'close': 'price'}, inplace=True)
    
    price_folder = r"C:\Users\YuweiCao\Documents\GitHub\Project\Project\etherscan\price\\"
    selected_columns.to_csv(price_folder + f"{chain_name}_price.csv", index=False)
    
chain_names = mapping.keys()
for chain_name in chain_names:
    fetch_price_data(chain_name)

Columns in DataFrame for ETH: Index(['low', 'updatedAt', 'timestamp', 'open', 'volume', 'high', 'close',
       'instrument'],
      dtype='object')
       low      updatedAt  timestamp     open    volume     high    close  \
0     2426  1705708745513 2024-01-18  2530.76  18838473  2549.89  2470.97   
1  2415.79  1705795002684 2024-01-19  2468.01  17179104  2505.95  2492.52   
2   2454.1  1705881462962 2024-01-20  2476.23   5103303     2491  2472.32   
3  2451.49  1705967900184 2024-01-21  2466.01   4309880  2482.58  2457.63   
4   2301.5  1706054331364 2024-01-22  2462.39  26475951  2466.99  2314.62   

                 instrument  
0  ETHUSDT-OKX-00000000-FUT  
1  ETHUSDT-OKX-00000000-FUT  
2  ETHUSDT-OKX-00000000-FUT  
3  ETHUSDT-OKX-00000000-FUT  
4  ETHUSDT-OKX-00000000-FUT  
Columns in DataFrame for TRX: RangeIndex(start=0, stop=0, step=1)
Empty DataFrame
Columns: []
Index: []
No valid data found for TRX. Skipping.
Columns in DataFrame for SOL: Index(['low', 'updatedAt', 'timesta

In [ ]:
import numpy as np

mapping = {
    'ETH': 'ETHUSDT-OKX-00000000-FUT',
    'SOL': 'SOLUSDT-OKX-00000000-FUT',
    'BTC': 'BTCUSDT-OKX-00000000-FUT',
}
def is_price_within_threshold(instrument, start_time,threshold):
    """
    判断最近三天的价格是否在阈值以内
    """
    primary_key_value = f"{instrument}|1"  # 假设使用 1440 分钟（日线）数据
    end_time = start_time - timedelta(days=1)
    price_data = price_bar_query(primary_key_value, start_time, end_time, env='live')

    if len(price_data) < 3:
        raise ValueError(f"价格数据不足三行，无法进行比较。")

    prices = price_data['close'].iloc[:3]

    # 计算价格差异平均值
    diff = np.mean(prices) -

    # 判断差异是否在阈值以内
    if diff <= threshold:
        return True
    else:
        return False

def classify_and_calculate_pnl(pnl_summary_df, mapping, price_bar_query, threshold=0.05, start_time=None, end_time=None):
    '''
    将 pnl_summary_df 中的 token 分为两类：主流币和其他币
    '''
    main_coin_results = []
    standalone_results = []

    for index, row in pnl_summary_df.iterrows():
        token = row["token"]
        price_unit = row["price_unit"]
        total_pnl = row["total_pnl"]
        
        # 判断是否含有中心币
        main_token = None
        for center_coin in mapping.keys():
            if re.search(fr"\b{center_coin}\b", token, re.IGNORECASE):
                main_token = center_coin
                break
        
        # 如果属于中心币，则判断 price_unit 是否正确
        if main_token:
            # 通过 price_bar_query 获取价格
            instrument = mapping[main_token]
            try:
                # 调用 price_bar_query 获取价格数据
                price_data = price_bar_query(instrument, start_time, end_time)
                if not price_data.empty:
                    # 获取对应的价格
                    actual_price = price_data["close"].iloc[-1]  # 获取最新价格
                    print(f"Actual price for {main_token}: {actual_price}")

                    # 判断 price_unit 是否包含主流币，并比较价格
                    if main_token in price_unit:
                        print(f"{token} matches {main_token} and price_unit is correct.")
                        main_coin_results.append({"token": main_token, "price_unit": price_unit, "total_pnl": total_pnl})
                    elif abs(actual_price - 1) <= threshold:  # 假设价格接近 1 表示稳定币
                        print(f"{token} matches {main_token} and price is within threshold. Adding to main coin results.")
                        main_coin_results.append({"token": main_token, "price_unit": price_unit, "total_pnl": total_pnl})
                    else:
                        print(f"{token} matches {main_token} but price or price_unit is incorrect.")
                        standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})
                else:
                    print(f"No price data found for {main_token}. Adding to standalone results.")
                    standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})
            except Exception as e:
                print(f"Error fetching price data for {main_token}: {e}")
                standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})
        else:
            print(f"{token} does not match any main token. Adding to standalone results.")
            standalone_results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})

    # 转换结果为 DataFrame
    main_coin_df = pd.DataFrame(main_coin_results)
    standalone_df = pd.DataFrame(standalone_results)

    # 打印汇总结果
    if not main_coin_df.empty:
        print("Main Coin PnL Results:")
        print(tabulate(main_coin_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
    else:
        print("No Main Coin PnL Results Found.")
    
    if not standalone_df.empty:
        print("Standalone PnL Results:")
        print(tabulate(standalone_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
    else:
        print("No Standalone PnL Results Found.")

    return main_coin_df, standalone_df
